```{glue:} glued_fig
```
# Cell nuclei detection using StarDist

In this notebook, we will use StarDist to detect cell nuclei in an image extracted from a public dataset of histopathology images.

[StarDist](https://github.com/stardist/stardist) is a deep-learning based Python library used for segmenting star-convex objects, such as cell nuclei, in 2D and 3D images. It is also available as plugins for [ImageJ](https://imagej.net/plugins/stardist), [Napari](https://github.com/stardist/stardist-napari), and [Qupath](https://qupath.readthedocs.io/en/0.3/docs/advanced/stardist.html).


In [ ]:
from myst_nb import glue
import panel as pn
pn.extension('plotly')

## Setup

Check that you have the `stardist` package installed (`pip install stardist`).

In [ ]:
from stardist.models import StarDist2D

## Download and read the image

The image we'll use in this tutorial is available for download on [Zenodo](https://zenodo.org/record/8099852) (`deepslide.png`). This image is part of the [DeepSlides](https://zenodo.org/record/1184621) public dataset.

In the cell below, we download this image from Zenodo and read it into a Numpy array.

In [ ]:
from shared_data import DATASET  # This dataset has a reference to the image on Zenodo.
from skimage.io import imread

image_file = DATASET.fetch("deepslide.png")

image = imread(image_file)

print(f'Loaded image in an array of shape: {image.shape} and data type {image.dtype}')
print(f'Intensity range: [{image.min()} - {image.max()}]')

## Visualize the image using matplotlib

Let's have a quick look at our example image.

In [ ]:
import matplotlib.pyplot as plt

fig_mpl, ax = plt.subplots(figsize=(8, 8))
ax.imshow(image, cmap='gray')
plt.title("H&E (DeepSlides)")
plt.axis('off')  # Hide axes ticks
plt.show()

## Intensity normalization

Let's rescale our image to the range 0-1. By doing so, it is also converted to an array of data type `float`.

In [ ]:
from skimage.exposure import rescale_intensity

image_normed = rescale_intensity(image, out_range=(0, 1))

print(f'Intensity range: [{image_normed.min()} - {image_normed.max()}]')
print(f'Array type: {image_normed.dtype}')

## Create a StarDist model

The StarDist developers provide a few pre-trained models that may already be applied to suitable images.

Here, we will use the *Versatile (H&E nuclei)* model that was trained on images from the MoNuSeg 2018 training data and the TNBC dataset from Naylor et al. (2018).

In [ ]:
model = StarDist2D.from_pretrained("2D_versatile_he")

model

## Run the model

We use the `predict_instances` method of the model to generate a segmenation mask (`labels`) and a representation of the cell nuclei as polygons (`polys`).

In [ ]:
labels, polys = model.predict_instances(
    image_normed,  # The image must be normalized
    axes="YXC",
    prob_thresh=0.5,  # Detection probability threshold
    nms_thresh=0.1,  # Remove detections overlapping by more than this threshold
    scale=1,  # Higher values are suitable for lower resolution data
    return_labels=True,
)

# We also get detection probabilities:
probabilities = list(polys["prob"])

n_detections = len(probabilities)

print(f'{n_detections} cells detected.')

## Visualization using Plotly

We display the segmentation overlaid on the image.

In [ ]:
import numpy as np
import plotly.express as px
import skimage.measure
import matplotlib

# Create a custom color lookup table based on detection probabilities
probas_incl_bg = np.zeros(n_detections + 1)  # Include a value for the background
probas_incl_bg[1:] = probabilities

fig = px.imshow(image, color_continuous_scale='gray')
    
for k, prob in zip(np.unique(labels), probabilities):
    col = (np.array(matplotlib.colormaps['inferno'](prob)) * 255).astype(int)
    ctr = skimage.measure.find_contours(labels == k)[0]
    cy, cx = np.mean(ctr, axis=0)
    y, x = ctr.T - 1
    fig.add_scatter(
        x=x,
        y=y,
        mode="lines",
        showlegend=False,
        line=dict(color=f'rgb({col[0]}, {col[1]}, {col[2]}, {col[3]})', width=1),
        hoverinfo="skip",
        opacity=1.0,
        fill="toself",
    )
    fig.add_scatter(
        x=[cx],
        y=[cy],
        mode='text',
        showlegend=False,
        text=[f'{prob:.02f}'],
        textposition="middle center",
        textfont=dict(family="Arial", size=12, color="white")
    )

fig.update_layout(
    title=f"StarDist result: {n_detections} cells detected.",
    width=600, 
    height=600, 
    margin=dict(t=50, l=0, b=0, r=0),
    xaxis=dict(showticklabels=False, ticks=""),
    yaxis=dict(showticklabels=False, ticks=""),
    plot_bgcolor='white'
)

glue('glued_fig', pn.pane.Plotly(fig), display=False)

```{glue:} glued_fig
```

## Conclusion

In this notebook, we have used `StarDist` to segment cell nuclei in an image from the DeepSlides dataset.

In [ ]:
fig.write_image("../images/stardist_plotly.jpeg")

![stardist](../../../images/stardist_plotly.jpeg)